In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import os
import ndjson
import re

In [2]:
pd.set_option('max_columns',200)

In [3]:
### directory in which data is located; user should change according to where they have put the data
DATA_DIR = '../../../variation-normalizer-manuscript-data'

In [4]:
#### change file path to location of clinvar normalizer output ndjson
with open(os.path.join(DATA_DIR,'output-variation_identity-all-liftover.ndjson'),'r') as f:
    records = ndjson.load(f)

In [5]:
len(records)

2210627

In [6]:
## read ClinVar normalizer responses into a pandas dataframe for analysis

batch_size = 100000
n_batches = len(records)//batch_size + 1

df0 = pd.concat([pd.json_normalize(records[k*batch_size:(k+1)*batch_size]) for k in tqdm(range(n_batches))])

  0%|          | 0/23 [00:00<?, ?it/s]

In [7]:
### re-run to re-initialize the ClinVar dataframe without re-running the above cell to read in the full dataframe
df = df0.copy()

#  Identify CNVs in ClinVar

In [8]:
### different predicted behavior of variants based on variant type/what information provided by ClinVar
df['in.vrs_xform_plan.policy'].value_counts()

Canonical SPDI                                      2118669
Absolute copy count                                   53263
Copy number change (cn loss|del and cn gain|dup)      27104
NCBI36 genomic only                                    4771
No hgvs or location info                               3089
Genotype/Haplotype                                     1440
Invalid/unsupported hgvs                               1336
Remaining valid hgvs alleles                            941
Min/max copy count range not supported                   14
Name: in.vrs_xform_plan.policy, dtype: int64

### Restrict to copy number variants. 
ClinVar variant types were further specified and identified in the ```in.vrs_xform_plan.policy``` field before running the normalizeer on ClinVar. 
For our analysis, CNVs are variants with ```in.vrs_xform_plan.policy``` equal to one of  
* ```Absolute copy count```
* ```Copy number change (cn loss|del and cn gain|dup)```
* ```Min/max copy count range not supported```

In [9]:
df = df[df['in.vrs_xform_plan.policy'].str.lower().str.contains('copy')].copy()

In [10]:
df['in.variation_type'].value_counts()

copy number loss    28887
copy number gain    26961
Deletion            16341
Duplication          8192
Name: in.variation_type, dtype: int64

In [11]:
len(df)

80381

### Remove variants which failed to normalize

In [12]:
### errors stored as a list of values, some of which are strings and other of which are dictionaries (determined by whether error was handled at the level of Variation Normalizer or after the normalizer)
### this function extracts the text error responses for better readability and ease string processing

def get_errors(errors):
    errors_out = []
    for e in errors:
        if type(e)== str:
            errors_out.append(e)
        elif type(e)==dict:
            for k,v in e.items():
                if k not in ['msg','response-errors']: ## only get these keys from normalizer response
                    continue
                if type(v) == str:
                    errors_out.append(v)
                elif type(v) == list:
                    errors_out.append(';'.join(v))
                #errors_out.append('[NORMALIZER] '+';'.join(e.get('response-errors',[])))
    return ';'.join(errors_out)


### to get the core error message, we'll simply strip all the numeric values out and replace them with "#"
def reduce_errors(error_string):
    out = error_string.lower()
    out = re.sub('\d+','#',out)
    return out

### an even further reduction to bin error strings into categories
def reduce_errors_again(error_string):
    outs = []
    for t in ['unable to tokenize','unable to find classification','unable to find a grch# accession for','not a supported hgvs genomic duplication or deletion']:
        if t in error_string:
            outs.append(t)
    if outs == []:
        return 'Success'
    return ';'.join(sorted(outs))

In [13]:
### apply the error extraction function to our dataframe's column containing the errors
df['error_string'] = df['out.errors'].fillna('').apply(get_errors)
df['error_string_reduce'] = df['error_string'].apply(reduce_errors)
df['error_string_reduce_2'] = df['error_string_reduce'].apply(reduce_errors_again)

In [14]:
df['error_string_reduce_2'].value_counts()

Success                                                 79361
unable to find a grch# accession for                      539
unable to find classification                             384
unable to find classification;unable to tokenize           95
not a supported hgvs genomic duplication or deletion        2
Name: error_string_reduce_2, dtype: int64

### Remove variants which failed to normalize 
Due to
* liftover error
* classification error
* tokenization error
* not supported by normalizer

In [15]:
df = df[df['error_string_reduce']==''].copy()

In [16]:
len(df)

79353

# Calculate start/stop positions to compare to NCH variants
### What start/stop position fields are available
* ```out.subject.start.value``` is the normalized start value when the start is specified (similarly for ```end```)
* If the start is a range, we look to ```out.subject.start.min``` and ```out.subject.start.max``` (similarly for ```end```)

### Our policy for choosing start/stop positions: Prioritize the absolute start and stop position (if known), otherwise inner, then outer 
* choose the specified normalized start value when available; otherwise the inner start (i.e. the max start); otherwise the outer start (i.e. the min start)
* similarly, choose the normalized end value when available; otherwise the inner stop (i.e. the max stop); otherwise the outer stop (i.e. the max stop)

In [17]:
df['start_38'] = df['out.subject.start.value'].fillna(df['out.subject.start.max']).fillna(df['out.subject.start.min']).astype(float)
df['stop_38'] = df['out.subject.end.value'].fillna(df['out.subject.end.min']).fillna(df['out.subject.end.max']).astype(float)

In [18]:
### Let's get a look at variants which do not have sufficient data to populate the start/stop position fields using our chosen preference rule

df[
    (df['start_38'].isna()) | (df['stop_38'].isna())
][['in.seq.assembly','in.hgvs.assembly','in.seq.start','in.seq.inner_start','in.seq.outer_start','out.subject.start.value','out.subject.start.max','out.subject.start.min','in.min_copies','in.max_copies']]

,in.seq.assembly,in.hgvs.assembly,in.seq.start,in.seq.inner_start,in.seq.outer_start,out.subject.start.value,out.subject.start.max,out.subject.start.min,in.min_copies,in.max_copies
48488,GRCh37,NaN,NaN,131243689,131225843,NaN,NaN,NaN,3,4
21314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99171,GRCh37,NaN,NaN,168483,NaN,NaN,NaN,NaN,1,2
68115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68121,GRCh37,NaN,NaN,22770421,NaN,NaN,NaN,NaN,3,4
89854,GRCh37,NaN,NaN,173786,NaN,NaN,NaN,NaN,2,3
90103,GRCh37,NaN,NaN,1,NaN,NaN,NaN,NaN,4,5
15670,GRCh37,NaN,NaN,22770421,NaN,NaN,NaN,NaN,3,4
61005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64340,GRCh37,NCBI36,NaN,151048,NaN,NaN,NaN,NaN,NaN,NaN


### Why do these variants not have normalized location information?
* 3 are in build 36 (not supported)
* 6 have no genome assembly specified
* 14 have a copy number range (not supported)

### Restrict to CNVs with calculated location information

In [19]:
df = df[df['start_38'].notna() & df['stop_38'].notna()].copy()
df['start_38'] = df['start_38'].astype(int)
df['stop_38'] = df['stop_38'].astype(int)
len(df)

79330

In [20]:
## restrict dataframe to minimal necessary fields, clean up column names for export and downstream analyses

df = df[['in.id','in.name','in.variation_type','in.seq.assembly','in.seq.chr','start_38','stop_38','in.absolute_copies','in.min_copies','in.max_copies']]
df = df.rename(columns = {c:c.split('.')[-1] for c in df.columns})
df

,id,name,variation_type,assembly,chr,start_38,stop_38,absolute_copies,min_copies,max_copies
159,1676567,NM_005445.4(SMC3):c.1343dup (p.Glu449fs),Duplication,NaN,NaN,110589641,110589642,NaN,NaN,NaN
169,29615,NC_000014.9:g.(53815591_53825260)_(53983697_53...,Deletion,NaN,NaN,53825259,53983698,NaN,NaN,NaN
172,1340493,GRCh37/hg19 12p12.3(chr12:17595624-18236175)x3,copy number gain,GRCh37,12,17442689,18083241,3,NaN,NaN
173,1340674,GRCh37/hg19 7q31.33-32.1(chr7:127050634-127826...,copy number gain,GRCh37,7,127410579,128186178,3,NaN,NaN
175,814630,GRCh37/hg19 4q33-34.1(chr4:171505226-173149981)x3,copy number gain,GRCh37,4,170584074,172228830,3,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
99240,866568,NM_015629.4(PRPF31):c.73_166dup (p.Asp56fs),Duplication,GRCh38,19,54118350,54118444,NaN,NaN,NaN
99289,847001,NM_000051.4(ATM):c.3273_3284+42dup,Duplication,GRCh38,11,108272840,108272894,NaN,NaN,NaN
10489,993055,NC_000013.10:g.32889619_32890666dup,Duplication,GRCh38,13,32315481,32316529,NaN,NaN,NaN
10492,1755741,NM_000059.4(BRCA2):c.6842-587_7007+2347dup,Duplication,GRCh38,13,32343970,32349243,NaN,NaN,NaN


In [21]:
df.to_csv(os.path.join(DATA_DIR,'ClinVar-CNVs-normalized.csv'),index=False)